In [1]:
import pandas as pd
data=pd.read_csv('output_data.csv')

d2 = data.dropna()

d2

,Packet Type,Packet Comments,TLS option 1,TLS option 2,TLS option 3,DNS TTL,DNS Domain Length,HTTP Content Type,HTTP Return Code,label
0,TLS,-1,264,73,-1,-1,-1,-1,-1,1
1,TLS,-1,1514,-1,49191,-1,-1,-1,-1,1
2,TLS,-1,220,-1,-1,-1,-1,-1,-1,1
3,TLS,-1,264,73,-1,-1,-1,-1,-1,1
4,TLS,-1,1514,-1,49191,-1,-1,-1,-1,1
...,...,...,...,...,...,...,...,...,...,...
1335377,TLS,-1,1514,-1,49171,-1,-1,-1,-1,1
1335378,TLS,-1,188,-1,-1,-1,-1,-1,-1,1
1335379,TLS,-1,194,41,-1,-1,-1,-1,-1,1
1335380,TLS,-1,1514,-1,49171,-1,-1,-1,-1,1


In [3]:
import pandas as pd
from multiprocessing import Pool
#ray.shutdown()
#ray.init()

#@ray.remote
def update_df(dframe1):
    dframe2 = pd.DataFrame(columns=[f'packet length {i}' for i in range(32)] + [f'tls option 1_{i}' for i in range(32)] + [f'tls option 2_{i}' for i in range(32)] + [f'dns option 1_{i}' for i in range(32)] + [f'dns option 2_{i}' for i in range(32)] + [f'http option 1_{i}' for i in range(32)] + [f'http option 2_{i}' for i in range(32)] + ['label'])
    row = {}
    for idx in range(len(dframe1)):
        if idx % 10000 == 0 and idx != 0:
            print("Continuing, count>> ", idx)
            dframe2.to_csv(f'./csv_results/csv_{idx}.csv', index = False)
            dframe2 = pd.DataFrame(columns=[f'packet length {i}' for i in range(32)] + [f'tls option 1_{i}' for i in range(32)] + [f'tls option 2_{i}' for i in range(32)] + [f'dns option 1_{i}' for i in range(32)] + [f'dns option 2_{i}' for i in range(32)] + [f'http option 1_{i}' for i in range(32)] + [f'http option 2_{i}' for i in range(32)] + ['label'])
        da = dframe1.iloc[idx]
        length = da['TLS option 1']
        len_bin = bin(length)
        bin_string = len_bin[2:].zfill(32)
        row.update({f'packet length {i}': int(bin_string[i]) for i in range(32)})
        packet_type = da['Packet Type']
        
        
        if packet_type == "TLS":
            if da['TLS option 2'] == -1 and da['TLS option 3'] == -1: 
                continue
            elif da['TLS option 2'] != -1:
                val = bin(int(da['TLS option 2']))
                bin_str = val[2:].zfill(32)
                row.update({f'tls option 1_{i}': int(bin_str[i]) for i in range(32)})
                row.update({f'tls option 2_{i}': -1 for i in range(32)})
            else:
                val = bin(int(da['TLS option 3']))
                #print(val)
                bin_str = val[2:].zfill(32)
                #print(bin_str)
                row.update({f'tls option 2_{j}': int(bin_str[j]) for j in range(32)})
                row.update({f'tls option 1_{i}': -1 for i in range(32)})
            row.update({f'dns option 1_{i}': -1 for i in range(32)})
            row.update({f'dns option 2_{i}': -1 for i in range(32)})
            row.update({f'http option 1_{i}': -1 for i in range(32)})
            row.update({f'http option 2_{i}': -1 for i in range(32)})
            
        elif packet_type == "DNS":
            val = bin(len(da['DNS TTL']))
            bin_str = val[2:].zfill(32)
            row.update({f'dns option 1_{i}': int(bin_str[i]) for i in range(32)})
            val = bin(int(da['DNS Domain Length']))
            bin_str = val[2:].zfill(32)
            row.update({f'dns option 2_{i}': int(bin_str[i]) for i in range(32)})
            row.update({f'tls option 1_{i}': -1 for i in range(32)})
            row.update({f'tls option 2_{i}': -1 for i in range(32)})
            row.update({f'http option 1_{i}': -1 for i in range(32)})
            row.update({f'http option 2_{i}': -1 for i in range(32)})
            
        elif packet_type == "HTTP":
            val = bin(len(da['HTTP Content Type']))
            bin_str = val[2:].zfill(32)
            row.update({f'http option 1_{i}': int(bin_str[i]) for i in range(32)})
            if da['HTTP Return Code'] == -1:
                row.update({f'http option 2_{i}': -1 for i in range(32)})
            else:
                val = bin(int(da['HTTP Return Code']))
                bin_str = val[2:].zfill(32)
                row.update({f'http option 2_{i}': int(bin_str[i]) for i in range(32)})
            row.update({f'tls option 1_{i}': -1 for i in range(32)})
            row.update({f'tls option 2_{i}': -1 for i in range(32)})
            row.update({f'dns option 1_{i}': -1 for i in range(32)})
            row.update({f'dns option 2_{i}': -1 for i in range(32)})
            
        else:
            continue
        
        row.update({'label':da['label']})
        
        dframe2 = dframe2.append(row, ignore_index=True)
        
        
    return dframe2

if __name__ == "__main__":
    #dframe1 = pd.read_csv('output_data.csv')
    pool = Pool(processes=8)
    results = pool.map(update_df, [d2])
    pool.close()
    pool.join()
    
    dframe2 = pd.concat(results)
    dframe2.to_csv('output_pleaseplz.csv', index = False)
    


Continuing, count>>  10000
Continuing, count>>  20000
Continuing, count>>  30000
Continuing, count>>  40000
Continuing, count>>  50000
Continuing, count>>  60000
Continuing, count>>  70000
Continuing, count>>  80000
Continuing, count>>  90000
Continuing, count>>  100000
Continuing, count>>  110000
Continuing, count>>  120000
Continuing, count>>  130000
Continuing, count>>  140000
Continuing, count>>  150000
Continuing, count>>  160000
Continuing, count>>  170000
Continuing, count>>  180000
Continuing, count>>  190000
Continuing, count>>  200000
Continuing, count>>  210000
Continuing, count>>  220000
Continuing, count>>  230000
Continuing, count>>  240000
Continuing, count>>  250000
Continuing, count>>  260000
Continuing, count>>  270000
Continuing, count>>  280000
Continuing, count>>  290000
Continuing, count>>  300000
Continuing, count>>  310000
Continuing, count>>  320000
Continuing, count>>  330000
Continuing, count>>  340000
Continuing, count>>  350000
Continuing, count>>  360000
C

In [4]:
import os
import glob

In [7]:
path = './csv_results/'

all_files = glob.glob(os.path.join(path,"*.csv"))


In [8]:
all_files


['./csv_results/csv_440000.csv',
 './csv_results/csv_330000.csv',
 './csv_results/csv_390000.csv',
 './csv_results/csv_670000.csv',
 './csv_results/csv_240000.csv',
 './csv_results/csv_260000.csv',
 './csv_results/csv_730000.csv',
 './csv_results/csv_last.csv',
 './csv_results/csv_520000.csv',
 './csv_results/csv_830000.csv',
 './csv_results/csv_880000.csv',
 './csv_results/csv_150000.csv',
 './csv_results/csv_510000.csv',
 './csv_results/csv_190000.csv',
 './csv_results/csv_30000.csv',
 './csv_results/csv_800000.csv',
 './csv_results/csv_810000.csv',
 './csv_results/csv_490000.csv',
 './csv_results/csv_470000.csv',
 './csv_results/csv_780000.csv',
 './csv_results/csv_640000.csv',
 './csv_results/csv_540000.csv',
 './csv_results/csv_210000.csv',
 './csv_results/csv_140000.csv',
 './csv_results/csv_450000.csv',
 './csv_results/csv_350000.csv',
 './csv_results/csv_590000.csv',
 './csv_results/csv_630000.csv',
 './csv_results/csv_370000.csv',
 './csv_results/csv_720000.csv',
 './csv_resul

In [9]:
combined_data = pd.DataFrame()

In [10]:
for file in all_files:
    df = pd.read_csv(file)
    
    combined_data = pd.concat([combined_data, df], axis = 0)
    
    

In [11]:
combined_data.to_csv('combined_data.csv', index = False)

In [12]:
combined_data

,packet length 0,packet length 1,packet length 2,packet length 3,packet length 4,packet length 5,packet length 6,packet length 7,packet length 8,packet length 9,...,http option 2_23,http option 2_24,http option 2_25,http option 2_26,http option 2_27,http option 2_28,http option 2_29,http option 2_30,http option 2_31,label
0,0,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,0
1,0,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,0
2,0,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,0
3,0,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,0
4,0,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9072,0,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,0
9073,0,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,0
9074,0,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,0
9075,0,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,0


In [14]:
combined_data['label'].unique()

array([0, 1])

In [15]:
from sklearn.model_selection import train_test_split


In [16]:
train_data, test_valid_data = train_test_split(combined_data, test_size=0.3, random_state=1)

In [17]:
test_data, valid_data = train_test_split(test_valid_data, test_size=0.5, random_state=1)

In [18]:
from autogluon.tabular import TabularPredictor

/home/dnslab/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/dnslab/.local/lib/python3.8/site-packages/sklearn/utils/optimize.py:18: DeprecationWarning: Please use `line_search_wolfe2` from the `scipy.optimize` namespace, the `scipy.optimize.linesearch` namespace is deprecated.
  from scipy.optimize.linesearch import line_search_wolfe2, line_search_wolfe1
/home/dnslab/.local/lib/python3.8/site-packages/sklearn/utils/optimize.py:18: DeprecationWarning: Please use `line_search_wolfe1` from the `scipy.optimize` namespace, the `scipy.optimize.linesearch` namespace is deprecated.
  from scipy.optimize.linesearch import line_search_wolfe2, line_search_wolfe1


In [19]:
target_column = "label"

In [21]:
import time

In [22]:
start = time.time()
predictor = TabularPredictor(label='label').fit(train_data=train_data, tuning_data = valid_data)
end = time.time()

No path specified. Models will be saved in: "AutogluonModels/ag-20230425_162055/"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20230425_162055/"
AutoGluon Version:  0.1.0
Train Data Rows:    563320
Train Data Columns: 224
Tuning Data Rows:    120712
Tuning Data Columns: 224
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    122498.4 MB
	Train Data (Original)  Memory Usage: 1225.79 MB (1.0% of available memory)
	Inferring data type of each feature based on column values. S

In [23]:
print('training time: '+str(end - start))
predictions = predictor.predict(test_data)

training time: 7956.461863040924


In [26]:
print(predictions)
score = predictor.evaluate(test_data)

2740    1
6851    0
1195    0
4448    1
5951    0
       ..
7321    0
8086    0
7104    1
902     0
4879    1
Name: label, Length: 120712, dtype: int64
Predictive performance on given data: accuracy = 0.9538985353568825


KeyError: 8985